#Máquinas de Boltzmann Restringidas

# Instalamos pytorch

In [5]:
pip install torch===1.6.0 torchvision===0.7.0 -f https://download.pytorch.org/whl/torch_stable.html

Defaulting to user installation because normal site-packages is not writeableNote: you may need to restart the kernel to use updated packages.

Looking in links: https://download.pytorch.org/whl/torch_stable.html


ERROR: Could not find a version that satisfies the requirement torch===1.6.0 (from versions: 1.7.1, 1.7.1+cpu, 1.7.1+cu101, 1.7.1+cu110, 1.8.0, 1.8.0+cpu, 1.8.0+cu101, 1.8.0+cu111, 1.8.1, 1.8.1+cpu, 1.8.1+cu101, 1.8.1+cu102, 1.8.1+cu111, 1.9.0, 1.9.0+cpu, 1.9.0+cu102, 1.9.0+cu111, 1.9.1, 1.9.1+cpu, 1.9.1+cu102, 1.9.1+cu111, 1.10.0, 1.10.0+cpu, 1.10.0+cu102, 1.10.0+cu111, 1.10.0+cu113, 1.10.1, 1.10.1+cpu, 1.10.1+cu102, 1.10.1+cu111, 1.10.1+cu113, 1.10.2, 1.10.2+cpu, 1.10.2+cu102, 1.10.2+cu111, 1.10.2+cu113, 1.11.0, 1.11.0+cpu, 1.11.0+cu113, 1.11.0+cu115, 1.12.0, 1.12.0+cpu, 1.12.0+cu113, 1.12.0+cu116, 1.12.1, 1.12.1+cpu, 1.12.1+cu113, 1.12.1+cu116, 1.13.0, 1.13.0+cpu, 1.13.0+cu116, 1.13.0+cu117, 1.13.1, 1.13.1+cpu, 1.13.1+cu116, 1.13.1+cu117, 2.0.0, 2.0.0+cpu, 2.0.0+cu117, 2.0.0+cu118, 2.0.1, 2.0.1+cpu, 2.0.1+cu117, 2.0.1+cu118, 2.1.0, 2.1.0+cpu, 2.1.0+cu118, 2.1.0+cu121, 2.1.1, 2.1.1+cpu, 2.1.1+cu118, 2.1.1+cu121, 2.1.2, 2.1.2+cpu, 2.1.2+cu118, 2.1.2+cu121)
ERROR: No matching distribut

In [8]:
pip install torch

Defaulting to user installation because normal site-packages is not writeable
     -------------------------------------- 192.2/192.2 MB 3.7 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [9]:
# Importar las librerías
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.optim as optim
import torch.utils.data
from torch.autograd import Variable

# Importar el dataset

In [11]:
movies = pd.read_csv("ml-1m/movies.dat", sep = '::', header = None, engine = 'python', encoding = 'latin-1')
users  = pd.read_csv("ml-1m/users.dat", sep = '::', header = None, engine = 'python', encoding = 'latin-1')
ratings  = pd.read_csv("ml-1m/ratings.dat", sep = '::', header = None, engine = 'python', encoding = 'latin-1')


# Preparar el conjunto de entrenamiento y elconjunto de testing

In [13]:
training_set = pd.read_csv("ml-100k/u1.base", sep = "\t", header = None)
training_set = np.array(training_set, dtype = "int")
test_set = pd.read_csv("ml-100k/u1.test", sep = "\t", header = None)
test_set = np.array(test_set, dtype = "int")

# Obtener el número de usuarios y de películas

In [14]:
nb_users = int(max(max(training_set[:, 0]), max(test_set[:,0])))
nb_movies = int(max(max(training_set[:, 1]), max(test_set[:, 1])))

# Convertir los datos en un array X[u,i] con usuarios u en fila y películas i en columna

In [15]:
def convert(data):
    new_data = []
    for id_user in range(1, nb_users+1):
        id_movies = data[:, 1][data[:, 0] == id_user]
        id_ratings = data[:, 2][data[:, 0] == id_user]
        ratings = np.zeros(nb_movies)
        ratings[id_movies-1] = id_ratings
        new_data.append(list(ratings))
    return new_data

In [16]:
training_set = convert(training_set)
test_set = convert(test_set)


# Convertir los datos a tensores de Torch

In [17]:
training_set = torch.FloatTensor(training_set)
test_set = torch.FloatTensor(test_set)

# Convertir las valoraciones a valores binarios 1 (Me gusta) o 0 (No me gusta)

In [18]:
training_set[training_set == 0] = -1
training_set[training_set == 1] = 0
training_set[training_set == 2] = 0
training_set[training_set >= 3] = 1

In [19]:
test_set[test_set == 0] = -1
test_set[test_set == 1] = 0
test_set[test_set == 2] = 0
test_set[test_set >= 3] = 1

# Crear la arquitectura de la Red Neuronal (Modelo Probabilistico Gráfico)

In [20]:
class RBM():
    def __init__(self, nv, nh):
        self.W = torch.randn(nh, nv)
        self.a = torch.randn(1, nh)
        self.b = torch.randn(1, nv)
    def sample_h(self, x):           #x = mini_batch_size x nv
        wx = torch.mm(x, self.W.t()) #mini_batch_size x nh
        activation = wx + self.a.expand_as(wx)
        p_h_given_v = torch.sigmoid(activation)
        return p_h_given_v, torch.bernoulli(p_h_given_v)
    def sample_v(self, y):           #y = mini_batch_size x nh
        wy = torch.mm(y, self.W) #mini_batch_size x nv
        activation = wy + self.b.expand_as(wy)
        p_v_given_h = torch.sigmoid(activation)
        return p_v_given_h, torch.bernoulli(p_v_given_h)   
    def train(self, v0, vk, ph0, phk):
        self.W += (torch.mm(v0.t(), ph0) - torch.mm(vk.t(), phk)).t()
        self.b += torch.sum((v0 - vk), 0)
        self.a += torch.sum((ph0 - phk), 0)

In [21]:
nv = len(training_set[0])
nh = 100
batch_size = 100

rbm = RBM(nv, nh)

# Entrenar la RBM


In [22]:
nb_epoch = 10
for epoch in range(1, nb_epoch+1):
    training_loss = 0
    s = 0.
    for id_user in range(0, nb_users - batch_size, batch_size):
        vk = training_set[id_user:id_user+batch_size]
        v0 = training_set[id_user:id_user+batch_size]
        ph0,_ = rbm.sample_h(v0)
        for k in range(10):
            _,hk = rbm.sample_h(vk)
            _,vk = rbm.sample_v(hk)
            vk[v0 < 0] = v0[v0 < 0]
        phk,_ = rbm.sample_h(vk)
        rbm.train(v0, vk, ph0, phk)
        training_loss += torch.mean(torch.abs(v0[v0>=0] - vk[v0>=0]))
        s += 1.
    print("Epoch: "+str(epoch)+", Loss: "+str(training_loss/s))

Epoch: 1, Loss: tensor(0.3298)
Epoch: 2, Loss: tensor(0.2499)
Epoch: 3, Loss: tensor(0.2480)
Epoch: 4, Loss: tensor(0.2500)
Epoch: 5, Loss: tensor(0.2446)
Epoch: 6, Loss: tensor(0.2520)
Epoch: 7, Loss: tensor(0.2463)
Epoch: 8, Loss: tensor(0.2444)
Epoch: 9, Loss: tensor(0.2465)
Epoch: 10, Loss: tensor(0.2465)


# Testear la RBM

In [23]:
testing_loss = 0
s = 0.
for id_user in range(nb_users):
    v = training_set[id_user:id_user+1]
    vt = test_set[id_user:id_user+1]
    if len(vt[vt>=0]) > 0:
        _,h = rbm.sample_h(v)
        _,v = rbm.sample_v(h)
        testing_loss += torch.mean(torch.abs(vt[vt>=0] - v[vt>=0]))
        s += 1.
        print("Testing Loss: "+str(testing_loss/s))


Testing Loss: tensor(0.2555)
Testing Loss: tensor(0.2868)
Testing Loss: tensor(0.2938)
Testing Loss: tensor(0.2703)
Testing Loss: tensor(0.2853)
Testing Loss: tensor(0.2691)
Testing Loss: tensor(0.2623)
Testing Loss: tensor(0.2510)
Testing Loss: tensor(0.2342)
Testing Loss: tensor(0.2253)
Testing Loss: tensor(0.2311)
Testing Loss: tensor(0.2215)
Testing Loss: tensor(0.2290)
Testing Loss: tensor(0.2227)
Testing Loss: tensor(0.2396)
Testing Loss: tensor(0.2370)
Testing Loss: tensor(0.2426)
Testing Loss: tensor(0.2334)
Testing Loss: tensor(0.2369)
Testing Loss: tensor(0.2387)
Testing Loss: tensor(0.2460)
Testing Loss: tensor(0.2482)
Testing Loss: tensor(0.2443)
Testing Loss: tensor(0.2434)
Testing Loss: tensor(0.2380)
Testing Loss: tensor(0.2391)
Testing Loss: tensor(0.2451)
Testing Loss: tensor(0.2417)
Testing Loss: tensor(0.2455)
Testing Loss: tensor(0.2522)
Testing Loss: tensor(0.2515)
Testing Loss: tensor(0.2518)
Testing Loss: tensor(0.2472)
Testing Loss: tensor(0.2429)
Testing Loss: 